#Downloading Audio from YouTube

In [ ]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pytube import YouTube

<h4>Paste the YouTube Link below :</h4>

In [ ]:
video_link = "https://www.youtube.com/watch?v=9frh7W_sPhI"    ### Paste the link here

In [ ]:
yt = YouTube(video_link)

In [ ]:
yt.streams.filter(only_audio = True, file_extension="mp4").first().download(filename = "ytaudio.mp4") 

'/content/ytaudio.mp4'

In [ ]:
!ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

#English ASR with HuggingSound

In [ ]:
!pip install huggingsound

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingsound import SpeechRecognitionModel

In [ ]:
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)

INFO:huggingsound.speech_recognition.model:Loading model...


#Audio Chunking 

In [ ]:
import librosa

In [ ]:
input_file = '/content/ytaudio.wav'

In [ ]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


#Speech to Text

In [ ]:
import soundfile as sf

In [ ]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

In [ ]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav') 

In [ ]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav']

In [ ]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 5/5 [00:02<00:00,  2.08it/s]


In [ ]:
full_transcript = " "
for item in transcriptions :
  full_transcript += ''.join(item['transcription']) 

In [ ]:
full_transcript

" good morning everyone respeted principal teachers and all-my deofriends tude am her to present a speech inferent of you on the topic self-confidencefirst ofall whate self-confidence self-confidence means having a belief in ourselves self-confidence cannot be taught by anyonit is something that we need to develop in oursells and it is also an important factor for a healthy and a posity lifestylewe all have different different goals in our life if we want to achieve this goals self-confidence is the major factorself-confidence makes a person independent optimistic and positive bynatureit doesn't mean that confident persons will always win and achievsuccess in their life but a self-confitent person can always able to face any difficult situation in their le confident people will always try to learn things from their mistakes and they never feel shy or feel in expressing their opinion in their lifeif whe are not enough confident we feel very difficult to say no to the things that we don'

In [ ]:
len(full_transcript)

1485

#Text Summarization

In [ ]:
from transformers import pipeline

In [ ]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
num_iters = int(len(full_transcript)/1000)
summarized_text = []
print('Summarized Text in Points is : \n')
for i in range (0,num_iters+1):
  start = 0
  start = i*1000
  end = (i+1)*1000
  out = summarization(full_transcript[start:end],min_length = 5, max_length = 20)
  out = out[0]
  out = out['summary_text']
  summarized_text.append(out)
  print("->" + out + "\n")

Summarized Text in Points is : 

-> Self-confidence means having a belief in ourselves . It is something that we need to

-> Self-confidence is not disimple thing that we can built or boost in ad



In [ ]:
# print(summarized_text)
output = ""
for x in summarized_text:
  output+=x+'.'
print(output)

 Self-confidence means having a belief in ourselves . It is something that we need to. Self-confidence is not disimple thing that we can built or boost in ad.
